# Real-Time Stream Viewer (HTTP)
the following function responds to HTTP requests with the list of last 10 processed twitter messages + sentiments in reverse order (newest on top), it reads records from the enriched stream, take the recent 10 messages, and reverse sort them. the function is using nuclio context to store the last results and stream pointers for max efficiency.<br> 

The code is automatically converted into a nuclio (serverless) function and and respond to HTTP requests<br>

the example demonstrate the use of `%nuclio` magic commands to specify environment variables, package dependencies,<br>configurations, and to deploy functions automatically onto a cluster.


## Initialize nuclio emulation, environment variables and configuration
use `# nuclio: ignore` for sections that don't need to be copied to the function

In [56]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [57]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

### Set function configuration 
use a cron trigger with 5min interval and define the base image<br>
for more details check [nuclio function configuration reference](https://github.com/nuclio/nuclio/blob/master/docs/reference/function-configuration/function-configuration-reference.md)

In [58]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


### Install required packages
`%nuclio cmd` allows you to run image build instructions and install packages<br>
Note: `-c` option will only install in nuclio, not locally

In [59]:
%nuclio cmd -c pip install v3io

## Nuclio function implementation
this function can run in Jupyter or in nuclio (real-time serverless)

In [60]:
import v3io.dataplane
import json
import os

def init_context(context):
    access_key = os.getenv('V3IO_ACCESS_KEY', None)
    setattr(context, 'container', os.getenv('V3IO_CONTAINER', 'users'))
    setattr(context, 'stream_path', os.getenv('STOCKS_STREAM',os.getenv('V3IO_USERNAME') + '/stocks/stocks_stream'))
    
    v3io_client = v3io.dataplane.Client(endpoint=os.getenv('V3IO_API', None), access_key=access_key)
    setattr(context, 'data', [])
    setattr(context, 'v3io_client', v3io_client) 
    setattr(context, 'limit', os.getenv('LIMIT', 10))
    

    
def handler(context, event):
    resp = context.v3io_client.seek_shard(container=context.container, path=f'{context.stream_path}/0', seek_type='EARLIEST')
    setattr(context, 'next_location', resp.output.location)
    resp = context.v3io_client.get_records(container=context.container, path=f'{context.stream_path}/0', location=context.next_location, limit=context.limit)
#     context.next_location = resp.output.next_location
    context.logger.info('location: %s', context.next_location)

    for rec in resp.output.records:
        rec_data = rec.data.decode('utf-8')
        rec_json = json.loads(rec_data)
        context.data.append({'Time': rec_json['time'],
                             'Symbol': rec_json['symbol'],
                             'Sentiment': rec_json['sentiment'],
                             'Link': rec_json['link'],
                             'Content': rec_json['content']})

    context.data = context.data[-context.limit:]
    
    columns = [{'text': key, 'type': 'object'} for key in ['Time', 'Symbol', 'Sentiment', 'Link', 'Content']]
    data = [list(item.values()) for item in context.data]
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response              

In [61]:
# nuclio: end-code

## Function invocation
the following section simulates nuclio function invocation and will emit the function results

In [62]:
# create a test event and invoke the function locally
init_context(context)
event = nuclio.Event(body='')
resp = handler(context, event)

Python> 2021-03-25 14:01:20,229 [info] location: AQAAAGYAAABHAEBeFwAAAA==


## Deploy a function onto a cluster
the `%nuclio deploy` command deploy functions into a cluster, make sure the notebook is saved prior to running it !<br>check the help (`%nuclio help deploy`) for more information

In [64]:
import os

In [65]:
from mlrun import code_to_function

# Export the bare function
fn = code_to_function('stream-viewer',
                      handler='handler')
fn.export('03-stream-viewer.yaml')

# Set parameters for current deployment
fn.set_envs({'V3IO_CONTAINER': 'users',
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') + '/stocks/stocks_stream'})
fn.spec.max_replicas = 2

> 2021-03-25 14:02:06,405 [info] function spec saved to path: 03-stream-viewer.yaml


In [66]:
project_name = "stocks-" + os.getenv('V3IO_USERNAME')
addr = fn.deploy(project=project_name)

> 2021-03-25 14:02:06,411 [info] Starting remote function deploy
2021-03-25 14:02:06  (info) Deploying function
2021-03-25 14:02:06  (info) Building
2021-03-25 14:02:06  (info) Staging files and preparing base images
2021-03-25 14:02:06  (info) Building processor image
2021-03-25 14:02:08  (info) Build complete
2021-03-25 14:02:12  (info) Function deploy complete
> 2021-03-25 14:02:13,174 [info] function deployed, address=default-tenant.app.dev8.lab.iguazeng.com:32536


In [ ]:
# nuclio: ignore
# test the new API end point, take the address from the deploy log above
!curl {addr}